In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import os 
import string
os.getcwd()

# Path to data file
# Note that you may need to add an extra slash ("\") in front of the existing slashes to avoid errors
data_path = Path("C:\\Users\\jsoto\\Desktop\\Mapping Data Workflow\\Mapping Directory for Spatial Join and Statistics\\Delta Mapping 2022\\Spatial Join\\T4_DeltaMapping_May2022_HR_spatialJoin.csv")
df_data = pd.read_csv(data_path)
df_data.head()


,Timestamp (PST),Latitude,Longitude,fDOM (QSU) (WS) HR,fCHL (μg/L) (WS) HR,NO3 (μM) (SUNA) HR,NO3 (mg-N/L) (SUNA) HR,Tr (%) (CStar) HR,C (1/m) (CStar) HR,Temp (˚C) (TSG) HR,...,estDIN (μM) HR,index_right,Poly_ID_01,Subregions,Location,Poly_ID_1,Poly_ID_5,Shape_Leng,Shape_Area,Resolution
0,2022-05-16 07:26:30,38.150235,-121.688935,43.971950,3.936434,18.705,0.2620,0.325338,11.228907,18.61115,...,NaN,1508.0,A682,Western Delta Tidal Zone,Sacramento River,B69,C14,0.019284,0.000012,0.1
1,2022-05-16 07:26:31,38.150240,-121.688920,43.950430,3.938943,18.670,0.2615,0.325288,11.230442,18.61190,...,NaN,1508.0,A682,Western Delta Tidal Zone,Sacramento River,B69,C14,0.019284,0.000012,0.1
2,2022-05-16 07:26:32,38.150245,-121.688915,43.957675,3.936434,18.705,0.2620,0.325338,11.228907,18.61240,...,NaN,1508.0,A682,Western Delta Tidal Zone,Sacramento River,B69,C14,0.019284,0.000012,0.1
3,2022-05-16 07:26:33,38.150250,-121.688910,43.950430,3.933924,18.740,0.2625,0.325288,11.230442,18.61290,...,NaN,1508.0,A682,Western Delta Tidal Zone,Sacramento River,B69,C14,0.019284,0.000012,0.1
4,2022-05-16 07:26:34,38.150255,-121.688905,43.941896,3.929252,18.770,0.2629,0.325338,11.228907,18.61345,...,NaN,1508.0,A682,Western Delta Tidal Zone,Sacramento River,B69,C14,0.019284,0.000012,0.1


In [2]:
# Lets see what columns we have
df_data.columns.values

array(['Timestamp (PST)', 'Latitude', 'Longitude', 'fDOM (QSU) (WS) HR',
       'fCHL (μg/L) (WS) HR ', 'NO3 (μM) (SUNA) HR',
       'NO3 (mg-N/L) (SUNA) HR', 'Tr (%) (CStar) HR',
       'C (1/m) (CStar) HR', 'Temp (˚C) (TSG) HR',
       'Cond (μS/cm) (TSG) HR', 'Salinity (PSU) (TSG) HR',
       'Temp (˚C) (EXO) HR', 'Sp Cond (μS/cm) (EXO) HR',
       'Salinity (PSU) (EXO) HR', 'pH (EXO) HR', 'pH (mV) (EXO) HR',
       'DO (mg/L) (EXO) HR', 'DO (%sat) (EXO) HR', 'Turb (FNU) (EXO) HR',
       'fDOM (QSU) (EXO) HR', 'fCHL (μg/L) (EXO) HR',
       'fBGA (ug/L) (EXO) HR', 'Green Algae (μg/L) (FP) HR',
       'Bluegreen Algae (μg/L) (FP) HR', 'Diatoms (μg/L) (FP) HR',
       'Cryptophyta (μg/L) (FP) HR', 'Yellow substances (μg/L) (FP) HR',
       'Total Conc (μg/L) (FP) HR', 'Transmission (%) (FP) HR',
       'Temp Sample (˚C) (FP) HR', 'Green Algae (%) (FP) HR',
       'Bluegreen Algae (%) (FP) HR', 'Diatoms (%) (FP) HR',
       'Cryptophyta (%) (FP) HR', 'NH4 (μM) HR ', 'NH4 (μM) HR RMK',

In [3]:
# Make a variable for the polygon id column name
# Change this to the name you identified in the list above
poly_id_col = "Poly_ID_01"

# Remove nulls in "Poly_ID column"
print(len(df_data))
df_data = df_data.dropna(subset=[poly_id_col])
print(len(df_data))


98031
97189


In [4]:
# Now lets make a list of the ones we don't want to include
# I am just copy/pasting non-constituent columns from above into this list
cols_to_drop = ['Timestamp (PST)', 'Latitude', 'Longitude', 'NH4 (μM) HR RMK',
                'index_right', 'Subregions', 'Location', 'Shape_Leng', 'Shape_Area', 'Resolution']

df_data = df_data.drop(cols_to_drop, axis="columns")
df_data.columns.values

array(['fDOM (QSU) (WS) HR', 'fCHL (μg/L) (WS) HR ', 'NO3 (μM) (SUNA) HR',
       'NO3 (mg-N/L) (SUNA) HR', 'Tr (%) (CStar) HR',
       'C (1/m) (CStar) HR', 'Temp (˚C) (TSG) HR',
       'Cond (μS/cm) (TSG) HR', 'Salinity (PSU) (TSG) HR',
       'Temp (˚C) (EXO) HR', 'Sp Cond (μS/cm) (EXO) HR',
       'Salinity (PSU) (EXO) HR', 'pH (EXO) HR', 'pH (mV) (EXO) HR',
       'DO (mg/L) (EXO) HR', 'DO (%sat) (EXO) HR', 'Turb (FNU) (EXO) HR',
       'fDOM (QSU) (EXO) HR', 'fCHL (μg/L) (EXO) HR',
       'fBGA (ug/L) (EXO) HR', 'Green Algae (μg/L) (FP) HR',
       'Bluegreen Algae (μg/L) (FP) HR', 'Diatoms (μg/L) (FP) HR',
       'Cryptophyta (μg/L) (FP) HR', 'Yellow substances (μg/L) (FP) HR',
       'Total Conc (μg/L) (FP) HR', 'Transmission (%) (FP) HR',
       'Temp Sample (˚C) (FP) HR', 'Green Algae (%) (FP) HR',
       'Bluegreen Algae (%) (FP) HR', 'Diatoms (%) (FP) HR',
       'Cryptophyta (%) (FP) HR', 'NH4 (μM) HR ', 'estDIN (μM) HR',
       'Poly_ID_01', 'Poly_ID_1', 'Poly_ID_5'], dty

In [5]:
# Compute Stats
# First group the data by poly_id_col
df_grouped = df_data.groupby(["Poly_ID_01", "Poly_ID_1", "Poly_ID_5"])

# Now lets define the statistics we want to compute in a list that we can pass to the pandas aggregation function
# For more information on what can go into this list check out: 
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.agg.html
#KO: added std to stats list 5/19/2020

stats = [
    "min",
    "max",
    "mean",
    "median",
    "std"
]

# Compute the statistics defined above for each polygon
df_stats = df_grouped.agg(stats)

# Flatten the hierarchical columns
df_stats.columns = [' '.join(col).strip() for col in df_stats.columns.values]

df_stats.head()


,,,fDOM (QSU) (WS) HR min,fDOM (QSU) (WS) HR max,fDOM (QSU) (WS) HR mean,fDOM (QSU) (WS) HR median,fDOM (QSU) (WS) HR std,fCHL (μg/L) (WS) HR min,fCHL (μg/L) (WS) HR max,fCHL (μg/L) (WS) HR mean,fCHL (μg/L) (WS) HR median,fCHL (μg/L) (WS) HR std,...,NH4 (μM) HR min,NH4 (μM) HR max,NH4 (μM) HR mean,NH4 (μM) HR median,NH4 (μM) HR std,estDIN (μM) HR min,estDIN (μM) HR max,estDIN (μM) HR mean,estDIN (μM) HR median,estDIN (μM) HR std
Poly_ID_01,Poly_ID_1,Poly_ID_5,,,,,,,,,,,,,,,,,,,,,
A1000,B96,C19,54.011720,54.588587,54.256028,54.215483,0.191662,5.221859,5.241367,5.236563,5.236109,0.005596,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A1001,B96,C19,53.600889,54.401583,54.127960,54.149299,0.236384,5.210934,5.241367,5.231962,5.235220,0.010652,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A1002,B96,C19,52.937695,53.600889,53.089024,52.941998,0.267429,5.160048,5.210934,5.192208,5.192717,0.015718,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A1003,B95,C19,52.964859,53.553388,53.144278,53.102412,0.175847,5.191239,5.213012,5.208229,5.211201,0.007575,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A1004,B95,C19,53.292370,53.553388,53.479680,53.491928,0.068363,5.178961,5.211552,5.198479,5.203137,0.012920,...,3.274899,3.429592,3.362009,3.369001,0.057978,37.652674,38.224592,38.009509,38.110021,0.23102


In [ ]:
# run the loop desired number of times. final output file will have the stats based on the final loop. 
# adjust number of iterations to make sure values converge using printed stat below
num_iterations = 12
tmp_df_data = df_data.copy() #copy of original df_data to preserve data
tmp_df_stats = df_stats.copy() 
for num in range(num_iterations):
    
    # merge original data and stats dataframes on poly ID
    df_merged = df_data.merge(tmp_df_stats, how = 'outer', left_on = 'Poly_ID_01', right_on = 'Poly_ID_01')
    
    convergence = 0
    for col in df_data.columns:
        # skip location, poly_id, lat, long columns when filtering data
        if col in ('Timestamp (PST)', 'Latitude', 'Longitude', 'index_right', 'Subregions', 'Location', 'Shape_Leng', 
                   'Shape_Area', 'Resolution', 'Poly_ID_01', 'Poly_ID_1', 'Poly_ID_5', 'NH4 (μM) HR RMK'):
            continue
 
        # creates series to hold mean and standard deviation values
        col_mean = col + ' mean' 
        col_std = col + ' std' # standard deviation

        # calculates and creates a column for 2x std
        col_2xstd = 2*df_merged[col_std]

        # calculates the difference between the instantaneous value and the mean value of its corresponding polygon
        col_diff_mean = abs(df_merged[col] - df_merged[col_mean])

        # if the difference between the instantenous value and the mean is greater than 2xstd replace the value with a NAN
        remove_mask = col_diff_mean > col_2xstd
        df_merged[col][remove_mask] = np.nan   
        convergence += col_2xstd.sum()
    print("Sum of standard deviations converging on: ", convergence)

    # creates dataframe, df_data_filtered
    # dropping extra columns created during filtering
    # filtered data retained, and columns kept are based on columns that existed in df_data
    tmp_df_data = df_merged[df_data.columns]
    
    # Re-run stats on filtered data
    df_grouped_filtered_01 = tmp_df_data.groupby('Poly_ID_01')
    df_grouped_filtered_1 = tmp_df_data.groupby('Poly_ID_1')
    df_grouped_filtered_5 = tmp_df_data.groupby('Poly_ID_5')
    
    # Compute the statistics defined above for each polygon
    tmp_df_stats_01 = df_grouped_filtered_01.agg(stats)
    tmp_df_stats_1 = df_grouped_filtered_1.agg(stats)
    tmp_df_stats_5 = df_grouped_filtered_5.agg(stats)
    
    # Flatten the hierarchical columns
    tmp_df_stats_01.columns = [' '.join(col).strip() for col in tmp_df_stats_01.columns.values]
    tmp_df_stats_1.columns = [' '.join(col).strip() for col in tmp_df_stats_1.columns.values]
    tmp_df_stats_5.columns = [' '.join(col).strip() for col in tmp_df_stats_5.columns.values]

# change name of dataframe for clarity after processing
df_stats_filtered_01 = tmp_df_stats_01
df_stats_filtered_1 = tmp_df_stats_1
df_stats_filtered_5 = tmp_df_stats_5

# stats for each polygon based on final iteration of above loop
# Write output file
out_dir = "C:\\Users\\jsoto\Desktop\\Mapping Data Workflow\\Mapping Directory for Spatial Join and Statistics\\Delta Mapping 2022\\Statistics"
out_fname_01 = data_path.name.split(".")[0] + "_stats_01.csv"
out_path_01 = Path(out_dir, out_fname_01)

out_fname_1 = data_path.name.split(".")[0] + "_stats_1.csv"
out_path_1 = Path(out_dir, out_fname_1)

out_fname_5 = data_path.name.split(".")[0] + "_stats_5.csv"
out_path_5 = Path(out_dir, out_fname_5)

# Flatten the hierarchical columns
df_stats_filtered_01.columns = [''.join(col).strip() for col in df_stats_filtered_01.columns.values]
df_stats_filtered_1.columns = [''.join(col).strip() for col in df_stats_filtered_1.columns.values]
df_stats_filtered_5.columns = [''.join(col).strip() for col in df_stats_filtered_5.columns.values]

# Write the csv
df_stats_filtered_01.to_csv(out_path_01, encoding = "utf-8-sig")
df_stats_filtered_1.to_csv(out_path_1, encoding = "utf-8-sig")
df_stats_filtered_5.to_csv(out_path_5, encoding = "utf-8-sig")


C:\Users\jsoto\Anaconda3\envs\rmp-spatial-join-stats\lib\site-packages\ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Sum of standard deviations converging on:  14686935.47221737
Sum of standard deviations converging on:  14686935.47221737
Sum of standard deviations converging on:  14686935.47221737
Sum of standard deviations converging on:  14686935.47221737


In [ ]:
# use a left join to append lat, long values of delta polygon center points
# Path to center point file
# Note that you may need to add an extra slash ("\") in front of the existing slashes to avoid errors
data_point01_path = Path("C:\\Users\\jsoto\\DOI\\BGC Projects (v3) - Documents\\Mapping Data Workflow\\Mapping Directory for Spatial Join and Statistics\\Center Points xlsx\\DeltaCenterPoints_01.xlsx")
data_point1_path = Path("C:\\Users\\jsoto\\DOI\\BGC Projects (v3) - Documents\\Mapping Data Workflow\\Mapping Directory for Spatial Join and Statistics\\Center Points xlsx\\DeltaCenterPoints_1.xlsx")
data_point5_path = Path("C:\\Users\\jsoto\\DOI\\BGC Projects (v3) - Documents\\Mapping Data Workflow\\Mapping Directory for Spatial Join and Statistics\\Center Points xlsx\\DeltaCenterPoints_5.xlsx")

df_point01 = pd.read_excel(data_point01_path)
df_point1 = pd.read_excel(data_point1_path)
df_point5 = pd.read_excel(data_point5_path)

# Join calculated stats on polygon id and center point id to get lat, long 
df_join01 = pd.merge(left=df_stats_filtered_01, right=df_point01, how='left', left_on='Poly_ID_01', right_on='CL_ID')
df_join1 = pd.merge(left=df_stats_filtered_1, right=df_point1, how='left', left_on='Poly_ID_1', right_on='CL_ID')
df_join5 = pd.merge(left=df_stats_filtered_5, right=df_point5, how='left', left_on='Poly_ID_5', right_on='CL_ID')

# Remove punctuation
df_join01.columns = df_join01.columns.str.strip().str.replace('[^\w\s]', '')
df_join1.columns = df_join1.columns.str.strip().str.replace('[^\w\s]', '')
df_join5.columns = df_join5.columns.str.strip().str.replace('[^\w\s]', '')

# stats for each polygon based on final iteration of above loop joined with CL_ID
# Write output file
out_dir2 = "C:\\Users\\jsoto\Desktop\\Mapping Data Workflow\\Mapping Directory for Spatial Join and Statistics\\Delta Mapping 2022\\For Interpolation"
out_fname01 = data_path.name.split(".")[0] + "_stats_join_01.csv"
out_path01 = Path(out_dir2, out_fname01)

out_fname1 = data_path.name.split(".")[0] + "_stats_join_1.csv"
out_path1 = Path(out_dir2, out_fname1)

out_fname5 = data_path.name.split(".")[0] + "_stats_join_5.csv"
out_path5 = Path(out_dir2, out_fname5)

# Flatten the hierarchical columns
df_join01.columns = [''.join(col).strip() for col in df_join01.columns.values]
df_join1.columns = [''.join(col).strip() for col in df_join1.columns.values]
df_join5.columns = [''.join(col).strip() for col in df_join5.columns.values]

# Write the csv
df_join01.to_csv(out_path01, encoding = "utf-8-sig")
df_join1.to_csv(out_path1, encoding = "utf-8-sig")
df_join5.to_csv(out_path5, encoding = "utf-8-sig")